In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from shower_environment import Shower
from stable_baselines3 import PPO,A2C
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
import os

logdir = "logs"
os.makedirs(logdir, exist_ok=True)

In [3]:
env = Shower()

print("Random valid temperature:", env.observation_space.sample())
print("Random valid temperature change:", env.action_space.sample()-1)

print("\nA few random showers:")
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    terminated = False
    score = 0 
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        score+=reward
    print(f"Episode: {episode} Score: {score}")

Random valid temperature: 70
Random valid temperature change: 0

A few random showers:
Episode: 1 Score: -60
Episode: 2 Score: -60
Episode: 3 Score: -34
Episode: 4 Score: -60
Episode: 5 Score: -18


In [4]:
# required before you can step the environment
env.reset()

# sample action:
print("sample action:", env.action_space.sample())

# observation space shape:
print("observation space shape:", env.observation_space.shape)

# sample observation:
print("sample observation:", env.observation_space.sample())

env.close()

sample action: 0
observation space shape: ()
sample observation: 14


In [5]:
def setup_model(model_class):
    model = model_class('MlpPolicy', env, verbose=1, tensorboard_log=logdir)
    model_string = model.learn.__defaults__[2]
    models_dir = f"models/{model_string}"
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
    return model, model_string, models_dir

In [6]:
def train_and_save(model, model_string, models_dir):
    TIMESTEPS = 10_000 # how frequently we save the model

    # Create a CheckpointCallback to save the best model
    checkpoint_callback = CheckpointCallback(save_freq=TIMESTEPS, save_path=logdir, name_prefix=model_string)

    # Create an EvalCallback to evaluate and save the best model
    eval_callback = EvalCallback(env, best_model_save_path=f"models/{model_string}", log_path=logdir, eval_freq=10000)
    
    model.learn(total_timesteps=100000, reset_num_timesteps=False, tb_log_name=model_string, callback=[checkpoint_callback,eval_callback])
    
    #iters = 0
    #for _ in range(1,10):
    #    iters += 1
    #    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=model_string)    
    #    model.save(f"{models_dir}/{TIMESTEPS*iters}")

In [7]:
model_class = PPO
model, model_string, models_dir = setup_model(model_class)
train_and_save(model, model_string, models_dir)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs/PPO_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -38.3    |
| time/              |          |
|    fps             | 5849     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -29.1       |
| time/                   |             |
|    fps                  | 3853        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010033726 |
|    clip_fraction        | 0.0317      |
|    clip_range           | 0.2         

/opt/homebrew/Caskroom/miniconda/base/envs/sb3/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60.4        |
|    ep_rew_mean          | 45.1        |
| time/                   |             |
|    fps                  | 3152        |
|    iterations           | 6           |
|    time_elapsed         | 3           |
|    total_timesteps      | 12288       |
| train/                  |             |
|    approx_kl            | 0.008596817 |
|    clip_fraction        | 0.0696      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.94       |
|    explained_variance   | 0.43        |
|    learning_rate        | 0.0003      |
|    loss                 | 43.8        |
|    n_updates            | 50          |
|    policy_gradient_loss | -0.00837    |
|    value_loss           | 73.6        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60.4  

In [8]:
model_class = A2C
model, model_string, models_dir = setup_model(model_class)
train_and_save(model, model_string, models_dir)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs/A2C_0
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 60       |
|    ep_rew_mean        | -14.5    |
| time/                 |          |
|    fps                | 2677     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.08    |
|    explained_variance | -0.176   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.528   |
|    value_loss         | 2.41     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 60       |
|    ep_rew_mean        | 8.5      |
| time/                 |          |
|    fps                | 2605     |
|    iterations         | 200    